# 社交媒体网络技术处理 —— 社交网络评论/博客模块

在本次的相关技术领域，我们尝试对项目结构进行模块化，用json格式文件进行读取，大致流程如下

先读一下 json

# 0 准备

In [29]:
import json
with open("intro.json","r",encoding="utf-8") as f:
    json_data = json.load(f)
username = json_data['username']
password = json_data['password']
webdriver_path = json_data['chrome_driver_location']
useremail = json_data['useremail']
data_store_location = json_data['data_store_location']
result_analyse_location = json_data['result_analyse_location']
result_picture = json_data['result_picture']
keyword = json_data['keyword']
keyword

'福岛 核'

## 1. 数据爬取

可自行选定社交媒体网站，例如豆瓣，微博，知乎，微信公众号，今日头条等等，

爬取小组选定分析主题相关的社交媒体文本评论数据（要求至少爬取10万条数据，请添加具体实现步骤、运行结果和相关截图）（1分）

我们通过面向对象式写法实现跨模块的调用技术

In [ ]:
from webSpyder.toutiaoSpyder import ToutiaoSpyder
toutiao = ToutiaoSpyder(webdriver_path,keyword)
toutiao.enter_key()
toutiao.getSpyNews()
toutiao.getSpyMicro()
toutiao.store_mongoDB()

# 知乎同理，进行数据获取时用到，爬取相关的评论和文章
from webSpyder.zhihuSpyder import ZhiHuSpyder
zhihu = ZhiHuSpyder(webdriver_path)
zhihu.getComments()
zhihu.getSpyContent()

# 其他模型不再赘述

## 爬取数据预处理

#### 1. 格式文本统一化

我们的数据来源比较复杂，有来着知乎、头条、微博的数据，存储格式也大有不同，
因此我们需要进行所有的数据归一化，同时执行相关的预处理操作

预处理模块的代码参考dataProcessing.py ，包括内部的一系列情况

In [30]:
import pandas as pd
import dataProcessing as dp

excelData = data_store_location+'/excelData'
mongoData = data_store_location+'/mongoData'
otherData = data_store_location+'/otherData'

excel = dp.multi_excel_combine(excelData)
mongo = dp.multi_csv_combine(mongoData)
other = dp.multi_csv_combine(otherData)
print(excel.shape)
print(mongo.shape)
print(other.shape)
print("whole raw data: ",excel.shape[0]+mongo.shape[0]+other.shape[0])

(18611, 12)
(264000, 4)
(55924, 13)
whole raw data:  338535


可以看到未处理原始评论数据集有20w条左右，下面对这些数据进行处理整合

In [31]:
# 集体去重,先处理微博的情况
df_weibo_excel = dp.data_renameweibo(excel)
df_weibo_other = dp.data_renameweibo(other)

# 对excel和other下的微博数据集体抽取并合并起来
dfA = dp.data_cut(df_weibo_excel)
dfB = dp.data_cut(df_weibo_other)
mg = dp.data_cut(mongo)

dfAB = dp.multi_pd_combine(dfA, dfB)
dfABC = dp.multi_pd_combine(dfAB, mg)
dfABC.head(10)

,author,content,time
0,weltncap,#福岛核废水57天可废掉半个太平洋#加速末日进程,04月13日 23:59
1,辛宪英亲卫队,//@司马南:在日本政府13日决定将福岛核废水排入海中之后，与其邻近的中国、韩国均已批评了日...,04月13日 23:59
2,曾文泉1,@VV深呼吸:可恶的日本鬼子！央视报导；日本内阁于今日通过往太平洋倾倒核废料决定、日本人想干...,04月13日 23:59
3,北纬27-,#福岛核废水57天可废掉半个太平洋#先把他们都毒死吧，清理地盘出来放核废水,04月13日 23:59
4,饼干小宝99,#日本核废水# 2厦门·厦门大学,04月13日 23:59
5,专业背锅你鲨爹,#福岛核废水57天可废掉半个太平洋#待那一天，海浪卷上你们的岛泽，吞没你们的房屋，你们连造出...,04月13日 23:59
6,小舟渡江,怎么会通过放出核废水？？？脑子没病吧,04月13日 23:59
7,她的独角兽,#福岛核废水57天可废掉半个太平洋#积点德吧，简直丧心病狂,04月13日 23:59
8,闷闷0088,#如何看待日本福岛核废水排海#.I'm not living to please you.,04月13日 23:59
9,哈哈赵是赵哈哈,#福岛核废水57天可废掉半个太平洋#看到这种事情，就觉得我们能做的就是珍惜当下且不要对未来过...,04月13日 23:59


pandas中，isnull()是常用的缺失值查看方法：

    df.isnull():查看所有数据是否为缺失值，返回bool值。True：缺失值；False：不是缺失值。
    df.isnull().any():查看每个特征是否存在缺失值，返回bool值。
    df.isnll().sum():返回每个特征中缺失值的具体数量。



In [32]:
dp.show_nan_data(dfABC)

dfABC=dfABC.dropna(axis=0, subset= ['content']).dropna(axis=0,subset=['author'])
dfABC.reset_index(drop=True,inplace=True)#drop=True：删除原行索引；inplace=True:在数据上进行更新

dfABC

author      465
content    1819
time          0
dtype: int64


,author,content,time
0,weltncap,#福岛核废水57天可废掉半个太平洋#加速末日进程,04月13日 23:59
1,辛宪英亲卫队,//@司马南:在日本政府13日决定将福岛核废水排入海中之后，与其邻近的中国、韩国均已批评了日...,04月13日 23:59
2,曾文泉1,@VV深呼吸:可恶的日本鬼子！央视报导；日本内阁于今日通过往太平洋倾倒核废料决定、日本人想干...,04月13日 23:59
3,北纬27-,#福岛核废水57天可废掉半个太平洋#先把他们都毒死吧，清理地盘出来放核废水,04月13日 23:59
4,饼干小宝99,#日本核废水# 2厦门·厦门大学,04月13日 23:59
...,...,...,...
336246,知乎用户,我一个准备去加国留学的看完心里怕怕的 惜命~_~,2017-02-13
336247,L-清源,如果短期旅游有什么影响吗？,2017-02-13
336248,黄sir,这个……事实上答主春节刚从日本旅游回来，还买了些化妆品 〒▽〒 不过仅限于去东京以东。不过大...,2017-02-12
336249,Siqi Xia (作者),天呐 请问哪些地方要注意不能去呢？我还准备四五月份去玩来着 去那玩还能吃海鲜那类吗？现在日本...,2017-02-12


In [33]:
dp.show_nan_data(dfABC)

author     0
content    0
time       0
dtype: int64


In [34]:
dfABC

,author,content,time
0,weltncap,#福岛核废水57天可废掉半个太平洋#加速末日进程,04月13日 23:59
1,辛宪英亲卫队,//@司马南:在日本政府13日决定将福岛核废水排入海中之后，与其邻近的中国、韩国均已批评了日...,04月13日 23:59
2,曾文泉1,@VV深呼吸:可恶的日本鬼子！央视报导；日本内阁于今日通过往太平洋倾倒核废料决定、日本人想干...,04月13日 23:59
3,北纬27-,#福岛核废水57天可废掉半个太平洋#先把他们都毒死吧，清理地盘出来放核废水,04月13日 23:59
4,饼干小宝99,#日本核废水# 2厦门·厦门大学,04月13日 23:59
...,...,...,...
336246,知乎用户,我一个准备去加国留学的看完心里怕怕的 惜命~_~,2017-02-13
336247,L-清源,如果短期旅游有什么影响吗？,2017-02-13
336248,黄sir,这个……事实上答主春节刚从日本旅游回来，还买了些化妆品 〒▽〒 不过仅限于去东京以东。不过大...,2017-02-12
336249,Siqi Xia (作者),天呐 请问哪些地方要注意不能去呢？我还准备四五月份去玩来着 去那玩还能吃海鲜那类吗？现在日本...,2017-02-12


可以看到缺失值已经被处理了，下面我们尝试去除重复值

pandas中，除去重复值的常用方法为drop_duplicate，其基本格式如下：

**DataFrame.drop_duplicates(subset=None, keep='first', inplace=False)：**

关键参数详解：

    subset:用来指定特定的列，默认所有列；
    keep: {'first', 'last', False}。默认值为'first'，用于删除重复项并保留第一次出现的项；
    inplace:是直接在原来数据上修改还是保留一个副本，默认是False

In [35]:
# 测试样例文档
dfN = dfABC.drop_duplicates(['content', 'author'],keep='last')
dfN.reset_index(drop=True,inplace=True)#drop=True：删除原行索引；inplace=True:在数据上进行更新

dfN

,author,content,time
0,心灵深处的呼唤1981,//@军粉姐姐V:美帝同意就往美帝那边倒呀，中国不同意！//@战略安全与军控在线:#日本政府...,04月13日 23:59
1,彭三岁的夫人呐,#福岛核废水57天可废掉半个太平洋#他怎么不稀释了自己喝,04月13日 23:59
2,Demiral28,//@失语卡文课代表:希望他的眼泪冲淡太平洋的核废水,04月13日 23:59
3,言小鸢,从麦穗没弯腰写起网购的猫草种子，据说是麦籽儿。一袋小麦，一袋大麦。它，边被我采摘与猫啃咬，边...,04月13日 23:59
4,手藝人陳大鍋,#福岛核废水57天可废掉半个太平洋#真的很可怕海洋动物到底做错啥子要承受这些？日本人大概率是...,04月13日 23:59
...,...,...,...
81841,知乎用户,我一个准备去加国留学的看完心里怕怕的 惜命~_~,2017-02-13
81842,L-清源,如果短期旅游有什么影响吗？,2017-02-13
81843,黄sir,这个……事实上答主春节刚从日本旅游回来，还买了些化妆品 〒▽〒 不过仅限于去东京以东。不过大...,2017-02-12
81844,Siqi Xia (作者),天呐 请问哪些地方要注意不能去呢？我还准备四五月份去玩来着 去那玩还能吃海鲜那类吗？现在日本...,2017-02-12


### 无效符号去除

下面进行去除或替换一些无效或不需要的信息，比如特殊字符、标点符号等。
请注意： 由于可能部分知乎的数据可能比较特殊，如“？？？？”这类的评论，去掉符号以后只有空的值了，因此需要再进行一次预处理，即再去掉横向空值，如果没有就OK

In [36]:
dfN['content'] = dfN['content'].str.replace(r'[^\w]+', '')
dfN.head()

<ipython-input-36-193a5cac010d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfN['content'] = dfN['content'].str.replace(r'[^\w]+', '')


,author,content,time
0,心灵深处的呼唤1981,军粉姐姐V美帝同意就往美帝那边倒呀中国不同意战略安全与军控在线日本政府正式决定福岛核废水排海...,04月13日 23:59
1,彭三岁的夫人呐,福岛核废水57天可废掉半个太平洋他怎么不稀释了自己喝,04月13日 23:59
2,Demiral28,失语卡文课代表希望他的眼泪冲淡太平洋的核废水,04月13日 23:59
3,言小鸢,从麦穗没弯腰写起网购的猫草种子据说是麦籽儿一袋小麦一袋大麦它边被我采摘与猫啃咬边顽强地成长现...,04月13日 23:59
4,手藝人陳大鍋,福岛核废水57天可废掉半个太平洋真的很可怕海洋动物到底做错啥子要承受这些日本人大概率是疯了吧...,04月13日 23:59


In [37]:
number = 0
for i in dfN['content'].values:
    if i == '':
        number+=1
print(number)

174


可以发现有这些数据content删除之后变成没有内容的假空值（但不是真空值 **"" != None**），但这类没有意义，也不适合作为重复值舍去

接下来我们对这类异常值进行预处理，直接把这一行的数据丢掉

In [39]:
import numpy as np
dfN['content'].replace("",np.nan,inplace=True)
dfN=dfN.dropna(axis=0, subset= ['content']).dropna(axis=0,subset=['author'])
dfN.reset_index(drop=True,inplace=True) #drop=True：删除原行索引；inplace=True:在数据上进行更新
dfN

,author,content,time
0,心灵深处的呼唤1981,军粉姐姐V美帝同意就往美帝那边倒呀中国不同意战略安全与军控在线日本政府正式决定福岛核废水排海...,04月13日 23:59
1,彭三岁的夫人呐,福岛核废水57天可废掉半个太平洋他怎么不稀释了自己喝,04月13日 23:59
2,Demiral28,失语卡文课代表希望他的眼泪冲淡太平洋的核废水,04月13日 23:59
3,言小鸢,从麦穗没弯腰写起网购的猫草种子据说是麦籽儿一袋小麦一袋大麦它边被我采摘与猫啃咬边顽强地成长现...,04月13日 23:59
4,手藝人陳大鍋,福岛核废水57天可废掉半个太平洋真的很可怕海洋动物到底做错啥子要承受这些日本人大概率是疯了吧...,04月13日 23:59
...,...,...,...
81667,知乎用户,我一个准备去加国留学的看完心里怕怕的惜命_,2017-02-13
81668,L-清源,如果短期旅游有什么影响吗,2017-02-13
81669,黄sir,这个事实上答主春节刚从日本旅游回来还买了些化妆品不过仅限于去东京以东不过大家不知道注意过没日...,2017-02-12
81670,Siqi Xia (作者),天呐请问哪些地方要注意不能去呢我还准备四五月份去玩来着去那玩还能吃海鲜那类吗现在日本零食已经...,2017-02-12


In [28]:
dfN[dfN['content'] == '为什么之前要后来不要了']

,author,content,time
72687,ChineseTea,为什么之前要后来不要了,04-22
